# Lab 2 - oczyszczanie danych

Dane w surowej formie nie zawsze nadają się do trenowania modeli predykcyjnych. Jedną z najczęstszych przyczyn zanieczyszczenia danych są brakujące wartości w poszczególnych atrybutach. Istnieją trzy popularne rozwiązania ,,walki" z wartościami brakującymi:
- usunięcie wierszy z wybrakowanymi wartościami,
- usunięcie całego atrybutu z wartościami wybrakowanymi,
- uzupełnienie wybranych wartości wybraną strategią.

Dobór rozwiązania w dużym stopniu zależy od charakteru zbioru danych oraz wartości wybrakowanych. W najbardziej idealnym scenariuszu należy sprawdzić efekt zapewniany przez wszystkie poznane rozwiązania, lecz w przypadku ograniczonych zasobów obliczeniowych może nie zawsze być to osiągalne. W praktyce można spotkać się z intyicyjnymi podejściami, które polegają na usuwaniu wierszy z brakującymi wartościami w przypadku ich niewielkiej liczebności, np. do ok. 0.1% oryginalnego zbioru danych. Podejście na usunięciu całego atrybutu sprawdza się gdy znaczna częśc wartości atrybutu jest wybrakowana. W pozostałych przypadkach najlepiej powinno sprawdzić się uzupełnianie wartości według obranej strategii. Wśród najbardziej typowych strategii uzupełniania wartości wybrakowanych warto wymienić następujące:
- średnia arytmetyczna lub mediana (dla wartości rzeczywistych),
- dominanta (dla danych kategorialnych).

Wśród popularnych podejść można znaleźć także związane z wytrenowanymi predyktorami:
- kNN: dopasowuje brakującą wartość na podstawie najbliższych sąsiadów pozostałych uzupełnionych wartości,
- regresor: dopasowuje brakującą wartość na podstawie modelu regresyjnego wytrenowanego na podstawie uzupełnionych wartości.

Wśród pozostałych metod warto zwrócić także uwagę na kontekst atrybutu, w którym występuje brakująca wartość. Przykładowo, jeżeli wybrakowana wartość dotyczy atrybutu poziomu leukocytów we krwi, warto sprawdzić jakie są normy, a następnie wylosować na podstawie wybranego rozkładu prawdopodobieństwa (np. gaussowskiego) wartość w przedziale stanowiącym normę populacji.

## Oczyszczanie danych na podstawie funkcjonalności biblioteki pandas

In [350]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)['frame']
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


Metoda *isnull* wywołana na obiekcie klasy Series zwróci serię wartości logicznych odpowiadających temu czy dana wartośc w kolumnie jest wybrakowana.

In [351]:
data['MedInc'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
20635    False
20636    False
20637    False
20638    False
20639    False
Name: MedInc, Length: 20640, dtype: bool

Za pomocą metody *any* wywołanej na powstałej w ten sposób ramce można sprawdzić czy występuje tam przynajmniej jedna wartość prawdziwa. Metoda all umożliwia sprawdzenie czy w ramce występują tylko i wyłącznie wartości prawdziwe.

In [352]:
data['MedInc'].isnull().any()

False

Za pomocą odwołania się do osi kolumn (parametr *axis=0* w metodzie *any*) można z łatwością sprawdzić w których kolumnach występują wartości wybrakowane.

In [353]:
data.isnull().any(axis=0)

MedInc         False
HouseAge       False
AveRooms       False
AveBedrms      False
Population     False
AveOccup       False
Latitude       False
Longitude      False
MedHouseVal    False
dtype: bool

Wiemy, że nasz dataset jest pełny. Spróbujmy zatem usunąć parę losowych wystąpień metodą chybił-trafił:

In [354]:
from random import randint

# minimalny i maksymalny odsetek komorek do usuniecia wartosci
min_percent, max_percent = 0.001, 0.003

# wyznacza pseudolosowo od min_percent do max_percent komórek z ramki danych
cells_to_remove = randint(int(data.size * min_percent), int(data.size * max_percent))

# pseudolosowy wybor indeksow wierszy i kolumn
for _ in range(cells_to_remove):
  row_idx = randint(0, data.shape[0] - 1)  # pseudolosowy indeks wiersza
  col_idx = randint(0, data.shape[1] - 1)  # pseudolosowy indeks kolumny

  # usuniecie pseudolosowo wskazanej komorki
  data.iat[row_idx, col_idx] = None

Oczywiścke istnieje takie prawdopodobieństwo, że "usuniemy" 2 bądź więcej razy tą samą komórkę, natomiast w tym przykładzie niezależy nam na unikalności usuwania.

Sprawdźmy zatem, jak teraz wygląda nasz dataset:

In [355]:
data.isnull().any(axis=0)

MedInc         True
HouseAge       True
AveRooms       True
AveBedrms      True
Population     True
AveOccup       True
Latitude       True
Longitude      True
MedHouseVal    True
dtype: bool

Jak widać udało nam się w każdej kolumnie uzyskać wartości brakujące. Doskonale!

Prześledźmy teraz metody uzupełniania wartości brakujących:

* Jeżeli w kolumnie występują wartości wybrakowane, można je uzupełnić wskazaną wartością.

In [356]:
data[data['MedInc'].isnull()].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
623,NaN,45.0,4.911475,1.026230,1003.0,3.288525,37.72,-122.18,1.567
624,NaN,45.0,5.377778,1.088889,519.0,3.844444,37.71,-122.18,1.575
1201,NaN,37.0,4.957447,1.053191,1505.0,3.202128,39.36,-121.70,0.560
1652,NaN,4.0,6.560729,0.939271,1552.0,3.141700,37.93,-121.97,3.070
1886,NaN,25.0,5.584071,1.296460,534.0,2.362832,38.93,-119.98,0.904


In [357]:
missing_indexes = data[data['MedInc'].isnull()].index # zapamiętujemy brakujące indeksy

In [358]:
data['MedInc'] = data['MedInc'].fillna(0) # uzupełniamy wszystko wartością 0

In [359]:
data.loc[missing_indexes].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
623,0.0,45.0,4.911475,1.026230,1003.0,3.288525,37.72,-122.18,1.567
624,0.0,45.0,5.377778,1.088889,519.0,3.844444,37.71,-122.18,1.575
1201,0.0,37.0,4.957447,1.053191,1505.0,3.202128,39.36,-121.70,0.560
1652,0.0,4.0,6.560729,0.939271,1552.0,3.141700,37.93,-121.97,3.070
1886,0.0,25.0,5.584071,1.296460,534.0,2.362832,38.93,-119.98,0.904


Aby umieścić wartości wybrakowane masowo w oryginalnej ramce danych należy użyć metody *fillna* na obiekcie klasy DataFrame przekazując jako parametr słownik mapujący nazwy kolumn na wartości, którymi mają zostać zastąpione wartości wybrakowane.

In [360]:
missing_indexes = data[(data['Longitude'].isnull()) | (data['Latitude'].isnull())].index

In [361]:
data.fillna({
    'Longitude': 0,
    'Latitude': 100,
}, inplace = True)

In [362]:
data.loc[missing_indexes].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
23,2.1806,52.0,5.193846,1.036923,853.0,2.624615,37.84,0.00,0.997
33,1.3750,49.0,5.030395,1.112462,754.0,2.291793,37.83,0.00,1.049
409,10.0825,52.0,8.209016,1.024590,658.0,2.696721,37.90,0.00,3.970
527,3.1771,52.0,4.827907,1.023256,482.0,2.241860,100.00,-122.25,2.102
595,4.8237,38.0,5.699387,1.007669,1784.0,2.736196,37.71,0.00,2.209


Biblioteka pandas dostarcza metod wyznaczających podstawowe (i bardziej zaawansowane) statystyki. Na szczególną uwagę zasługują metody *mean()* i *median()*, które zwracają odpowiednio średnią arytmetyczną i medianę wartości ze wskazanego atrybutu.

In [363]:
data['MedHouseVal'].mean()

2.06847767036065

In [364]:
data['MedHouseVal'].median()

1.797

W przypadku wyznaczania dominanty zastosowanie znajduje metoda *mode()*. Należy jednak pamiętać, że dominantą nie zawsze musi być tylko jedna wartość.

In [365]:
data['Population'].mode()

0    891.0
Name: Population, dtype: float64

In [366]:
data['Population'].mode().iloc[0]

891.0

## Oczyszczanie danych z biblioteką Scikit-learn

Oprócz biblioteki *pandas*, biblioteka *Scikit-learn* zawiera także obszerny zestaw narzędzi do pracy z oczyszczaniem danych, w szczególności z uzupełnianiem danych wybrakowanych. Przeznaczona do tego celu klasa *SimpleImputer* przyjmuje w inicjalizatorze parametr *strategy*, w którym należy wskazać metodę uzupełniania brakujących wartości:
- mean,
- median
- most_frequent,
- constant.

In [367]:
from sklearn.impute import SimpleImputer

In [368]:
imputer = SimpleImputer(strategy='median')

Przed właściwym uzupełnieniem wartości należy najpierw wybrać zestaw pasujących atrybutów, dla których zostanie zastosowana wybrana strategia. Pamiętaj, że używamy:
- średniej arytmetycznej lub mediany (dla wartości rzeczywistych),
- dominanty (dla danych kategorialnych).

In [369]:
num_attributes = data.select_dtypes(include=['number'])
num_attributes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [370]:
num_attributes.isnull().any(axis=0)

MedInc         False
HouseAge        True
AveRooms        True
AveBedrms       True
Population      True
AveOccup        True
Latitude       False
Longitude      False
MedHouseVal     True
dtype: bool

Wywołanie metody *fit* na utworzonym obiekcie pozwoli na automatyczne wyznaczenie wartości do uzupełnienia w każdym z atrybutów.

In [371]:
imputer.fit(num_attributes)

SimpleImputer(strategy='median')

W atrybucie *statistics_* zawarte są wyznaczone wartości do zastąpenia brakujących według obranej strategii.

In [372]:
imputer.statistics_

array([ 3.53130000e+00,  2.90000000e+01,  5.22822823e+00,  1.04878049e+00,
        1.16600000e+03,  2.81804949e+00,  3.42600000e+01, -1.18490000e+02,
        1.79700000e+00])

Do zastąpienia brakujących wartości przeznaczona jest metoda *transform*.

In [373]:
missing_indexes = num_attributes[num_attributes.isnull().any(axis = 1)].index
missing_indexes

Index([   96,   133,   151,   158,   179,   211,   264,   271,   286,   340,
       ...
       20241, 20245, 20355, 20370, 20373, 20382, 20415, 20441, 20499, 20533],
      dtype='int64', length=339)

In [374]:
new_num_attributes = imputer.transform(num_attributes)

In [375]:
import pandas as pd

In [376]:
new_num_attributes = pd.DataFrame(new_num_attributes, columns=data.columns)
new_num_attributes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [377]:
pd.DataFrame(imputer.statistics_.reshape((1,9)), columns= new_num_attributes.columns)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,3.5313,29.0,5.228228,1.04878,1166.0,2.818049,34.26,-118.49,1.797


In [378]:
new_num_attributes.loc[missing_indexes]

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
96,2.8345,31.0,3.894915,1.127966,2048.0,2.818049,37.82,-122.26,1.838
133,6.8976,34.0,6.612272,1.048780,1627.0,2.124021,37.83,-122.21,3.333
151,5.3509,52.0,5.650672,1.048780,1034.0,1.984645,37.81,-122.22,3.021
158,7.5080,52.0,7.994318,1.036932,878.0,2.494318,37.81,-122.23,1.797
179,2.9570,52.0,4.496259,1.047382,1321.0,2.818049,37.80,-122.24,1.228
...,...,...,...,...,...,...,...,...,...
20382,8.1064,29.0,7.337500,0.969643,1409.0,2.516071,34.14,0.00,4.234
20415,3.2176,16.0,4.226716,1.004902,1877.0,2.300245,34.17,-118.84,1.797
20441,6.0487,24.0,7.207080,1.030088,1166.0,3.246018,34.26,-118.78,2.545
20499,6.3199,23.0,6.909408,0.975610,978.0,3.407666,34.30,-118.71,1.797


In [379]:
new_num_attributes.isnull().any(axis=0)

MedInc         False
HouseAge       False
AveRooms       False
AveBedrms      False
Population     False
AveOccup       False
Latitude       False
Longitude      False
MedHouseVal    False
dtype: bool

Z uwagi na różnice między interfejsami bibliotek *pandas* i *Scikit-learn* warto zwrócić uwagę na typowe aspekty interfejsu aktualnie używanego narzędzia. Wykorzystywane są dwie niezależne metody: *fit* oraz *transform*. Wywołanie metody fit oznacza dopasowanie do aktualnie przekazanej ramki danych i wyznaczenie na jej podstawie wartości do uzupełnienia. Analogicznie wygląda sytuacja dla przekazanego atrybutu. Wywołanie metody *transform* spowoduje faktyczne uzupełnienie brakujących wartości i zwrócenie utworzonego w ten sposób nowego obiektu będącego pełną ramką danych. Warto więc mieć na uwadze, że próba wywołania metody *transform* po wywołaniu metody *fit* na innym zbiorze danych może kompletnie mijać się z celem.

## Zadania

1. Pobrać i wczytać zbiór danych danych dostępny pod adresem: https://archive.ics.uci.edu/dataset/10/automobile

2. Wartości wybrakowane w zbiorze oznaczone są symbolem "?". W celu zamiany na wartość *None* - dopasowaną do języka Python - można wykorzystać metodę *replace* wywoływaną na obiekcie ramki *pandas*. Utworzyć głęboką kopię powstałej w ten sposób ramki danych (funkcja [deepcopy](https://docs.python.org/3/library/copy.html)).

3. Zastosować poznane metody uzupełniania wartości wybrakowanych dopasowanych do typów danych w atrybutach za pomocą biblioteki *pandas*. W przypadku gdy do danego typu pasuje więcej niż jedna strategia (np. do typu ciągłego numerycznego: mediana i średnia arytmetyczna), utworzyć dwie wersje kolumn (np. col1\_mean i col1\_median), gdzie w każdej będą wartości uzupełnione inną strategią.

4. Na podstawie utworzonej kopii ramki z punktu 2, powtórzyć kroki z punktu 3, ale przy użyciu biblioteki *Scikit-learn*.

